# Feature extraction

## -1.0 Import the packages

In [ ]:
# on Docker/local
import sys
sys.path.append("../src")
sys.path.append("../DASstore") 
sys.path.append("/home/jmanos/notebooks/whidbey_noise")

import os
import time
from datetime import datetime, timedelta

import h5py
import numpy as np
import DAS_module
import matplotlib.pyplot as plt
from tqdm import tqdm
import dasquakes
from dasquakes import das_downloader

import importlib
importlib.reload(dasquakes)

import scipy
import scipy.signal
import seis_feature

## 0. Get the data

In [ ]:
cab = 'whidbey'
this_id = None
event_df = None
t0 = datetime(2023, 1, 18, 11, 0, 0)
record_length = 120


data, times, attrs, x_max, this_id, data_filt, t0 = das_downloader(event_df, this_id, cab, t0 = t0, record_length = record_length)

# start and end channel index for the sub-array
# cha1, cha2 = 0, attrs[ 'NumberOfLoci'] #all channels
cha1, cha2 = 1100, 1600 #some channels
cha_spacing = attrs['SpatialSamplingInterval']

In [ ]:
# Plot the data
plt.figure(figsize = (10, 5), dpi = 100)
plt.imshow(data_filt[:,cha1:cha2].T, aspect = 'auto', 
           cmap = 'RdBu', vmax = 1.5e1, vmin = -1.5e1, origin='lower')
_ =plt.yticks(np.linspace(cha1, cha2, 4) - cha1, 
              [int(i) for i in np.linspace(cha1, cha2, 4)], fontsize = 12)
plt.ylabel("Channel number", fontsize = 15)
plt.xlabel("Time", fontsize = 15)
twiny = plt.gca().twinx()
twiny.set_yticks(np.linspace(0, cha2 - cha1, 4), 
                             [int(i* cha_spacing) for i in np.linspace(cha1, cha2, 4)])
twiny.set_ylabel("Distance along cable (m)", fontsize = 15)
plt.colorbar(pad = 0.1)

# 1.0 Pre-process the data

In [ ]:
# detrend

detrended = scipy.signal.detrend(data[:,cha1:cha2].T, axis=1)

# fig,ax = plt.subplots()
# ax.imshow(detrended, aspect='auto', vmin=-1, vmax =1)

In [ ]:
# taper

window = scipy.signal.tukey(detrended.shape[-1], alpha=0.7)
taper = detrended * window

# fig,ax = plt.subplots()
# ax.imshow(taper, aspect='auto', vmin=-1, vmax =1)

In [ ]:
# Filtering
freqs = [[.001, .01],[.01, .1],[.1, 2]]
datas = []
for i in freqs:
    low_cut = i[0]
    hi_cut = i[1]
    

    b,a = scipy.signal.butter(2,[low_cut,hi_cut],'bp',fs=attrs['MaximumFrequency']*2,output='ba')
    data_filt = scipy.signal.filtfilt(b,a,taper,axis=1)
    datas.append(data_filt)

In [ ]:
# sliced=datas[1]
# fig,ax = plt.subplots()
# ax.imshow(sliced, aspect='auto', vmin=-np.percentile(sliced,99), vmax =np.percentile(sliced,99))

# 2.0 RSAM

## 2.1 Parameters to use

In [ ]:
fs=attrs['PulseRate']
wlen = int(6 * 60 * fs)
nmax = int(wlen*np.floor(data.shape[0]/wlen))

## 2.2 RSAM processing

In [ ]:
empty_list = []
for i in range(datas[0].shape[0]):
    datas_rsam = seis_feature.RSAM(datas[0][i], fs, empty_list, freqs[0], nmax, wlen) # get RSAM for different frequency bands

In [ ]:
# Plot the RSAM results
fig,ax = plt.subplots()
#ax.plot(np.asarray(empty_list).T)
ax.imshow(np.asarray(empty_list), aspect='auto')
plt.show()

## 3.0 Clustering

In [10]:
import os
import pickle

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import obspy
from obspy.clients.fdsn.client import Client 
import sklearn
from sklearn.decomposition import FastICA

from scatseisnet import ScatteringNetwork
%config InlineBackend.figure_format = "svg"

ContextualVersionConflict: (numpy 1.21.0 (/home/jmanos/miniconda3/envs/comcat/lib/python3.7/site-packages), Requirement.parse('numpy>=1.21.6'), {'scatseisnet'})